In [1]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
%matplotlib qt5

In [3]:
def plotImg(I, _3d = True, vmax=1):
    if _3d:
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})    

        X, Y = np.meshgrid(np.linspace(-1.0,1.0,I.shape[0]), np.linspace(-1.0,1.0, I.shape[1]))
        ax.plot_surface(X, Y, I, cmap=cm.coolwarm, linewidth=0, antialiased=False) 
    else:
        plt.imshow(I, vmin=0, vmax=vmax)
        
    plt.pause(0.0001)

In [4]:
def model(X, Y, O, framework):
    ux, uy, s21, p = O
    
    z1 = framework.exp( -( ( ((X-ux)**2.0)/s21 + ((Y-uy)**2.0)/s21 ) )**4.0 )
    
    return p*z1




def fitImg(img):

    X, Y = tf.meshgrid(np.linspace(-1.0,1.0,img.shape[0]), np.linspace(-1.0,1.0, img.shape[1]))


    y = model(X, Y, [0, 0, .5, 1], tf)
    y = img


    max_epoch = 2000
    lossFnc = tf.keras.losses.mean_squared_error
    optimizer = tf.keras.optimizers.SGD(learning_rate = 1.8)
    _VARIABLE = type(tf.Variable(0))

    ux  = tf.Variable(0.0, dtype=tf.float64)
    uy  = tf.Variable(0.0, dtype=tf.float64)
    s21  = tf.Variable(0.5, dtype=tf.float64)
    p   = tf.Variable(0.1, dtype=tf.float64)

    O         = [ux,   uy,   s21,  p]
    trainable = [True, True, True, True]

    # eta = 1.0
    for epoch in range(max_epoch):

        with tf.GradientTape(persistent=True) as tape:

            variables = [x[0] for x in zip(O, trainable) if x[1]] 

            tape.watch(variables)

            yHat = model(X, Y, O, tf)

            loss = tf.reduce_mean(lossFnc(y, yHat))

        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))

        if epoch%10 == 0:
    #         print([x.numpy() for x in O])
    #         print([x.numpy() for x in grads])
            print('batch : %d of %d (loss = %.4f)'%(epoch, max_epoch, loss))   
            plt.subplot(1,2,1)
            plt.cla()
            plotImg( y , False, 0.1)
            plt.subplot(1,2,2)
            plt.cla()
            plotImg( yHat , False, 0.1)  
            plt.pause(0.0001)

In [6]:
pwd

'/home/ic/particle-density/src'

In [7]:
#%% Read Image data
for ind in range(19):

    # Read image
    imfilename = '../results/10-microns particles-60X/crop_centered/crop_centered_' + str(ind) +'.tif'

    I = cv2.imread(imfilename, cv2.IMREAD_GRAYSCALE)

    #%% FIT GAUSSIAN
    new_shape = (32, 32)
    img = cv2.resize(I, new_shape, interpolation = cv2.INTER_AREA)
    
    bg = (np.mean(img[:4,:4]) + np.mean(img[:4,-4:]) + np.mean(img[-4:,:4]) + np.mean(img[-4:,-4:]))/4.0
    img = (img - bg)/255.0    
    
    fitImg(img)

batch : 0 of 2000 (loss = 0.0022)
batch : 10 of 2000 (loss = 0.0003)
batch : 20 of 2000 (loss = 0.0003)


<ipython-input-4-0fea57bbae14>:53: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.subplot(1,2,1)
<ipython-input-4-0fea57bbae14>:56: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.subplot(1,2,2)


batch : 30 of 2000 (loss = 0.0003)
batch : 40 of 2000 (loss = 0.0003)
batch : 50 of 2000 (loss = 0.0003)
batch : 60 of 2000 (loss = 0.0003)
batch : 70 of 2000 (loss = 0.0003)
batch : 80 of 2000 (loss = 0.0003)
batch : 90 of 2000 (loss = 0.0003)
batch : 100 of 2000 (loss = 0.0003)
batch : 110 of 2000 (loss = 0.0003)
batch : 120 of 2000 (loss = 0.0003)
batch : 130 of 2000 (loss = 0.0003)
batch : 140 of 2000 (loss = 0.0003)
batch : 150 of 2000 (loss = 0.0003)
batch : 160 of 2000 (loss = 0.0003)
batch : 170 of 2000 (loss = 0.0003)
batch : 180 of 2000 (loss = 0.0003)
batch : 190 of 2000 (loss = 0.0003)
batch : 200 of 2000 (loss = 0.0003)
batch : 210 of 2000 (loss = 0.0003)
batch : 220 of 2000 (loss = 0.0003)
batch : 230 of 2000 (loss = 0.0003)
batch : 240 of 2000 (loss = 0.0003)
batch : 250 of 2000 (loss = 0.0003)
batch : 260 of 2000 (loss = 0.0003)
batch : 270 of 2000 (loss = 0.0003)
batch : 280 of 2000 (loss = 0.0003)
batch : 290 of 2000 (loss = 0.0003)
batch : 300 of 2000 (loss = 0.0003)

In [ ]:
# 